# 编程作业 7 - K-means 和PCA（主成分分析）

在本练习中，我们将实现K-means聚类，并使用它来压缩图像。 我们将从一个简单的2D数据集开始，以了解K-means是如何工作的，然后我们将其应用于图像压缩。 我们还将对主成分分析进行实验，并了解如何使用它来找到面部图像的低维表示。

## K-means 聚类

我们将实施和应用K-means到一个简单的二维数据集，以获得一些直观的工作原理。 K-means是一个迭代的，无监督的聚类算法，将类似的实例组合成簇。 该算法通过猜测每个簇的初始聚类中心开始，然后重复将实例分配给最近的簇，并重新计算该簇的聚类中心。 我们要实现的第一部分是找到数据中每个实例最接近的聚类中心的函数。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from scipy.io import loadmat


In [ ]:
def find_closest_centroids(X, centroids):
# INPUT：数据X，初始聚类中心centroids
# OUTPUT：距离聚类中心最近的数据索引
# TODO：找到数据中每个实例最接近的聚类中心的数据
    
    # STEP1：得到矩阵的维度，初始化矩阵
    m = X.shape[0]
    k = centroids.shape[0]
    idx = np.zeros(m)
    
    # STEP2：遍历所有数据，找到距离聚类中心最近的
    # your code here  (appro ~ 3 lines)
    for i in range(m):
        min_dist = 1000000
        for j in range(k):
            dist = None
            if dist < min_dist:
                
                
    
    return idx

让我们来测试这个函数，以确保它的工作正常。 我们将使用练习中提供的测试用例。

In [ ]:
data = loadmat('data/ex7data2.mat')
X = data['X']
initial_centroids = initial_centroids = np.array([[3, 3], [6, 2], [8, 5]])

idx = find_closest_centroids(X, initial_centroids)
idx[0:3]

输出与文本中的预期值匹配（记住我们的数组是从零开始索引的，而不是从一开始索引的，所以值比练习中的值低一个）。 接下来，我们需要一个函数来计算簇的聚类中心。 聚类中心只是当前分配给簇的所有样本的平均值。

In [ ]:
data2 = pd.DataFrame(data.get('X'), columns=['X1', 'X2'])
data2.head()

In [ ]:
sb.set(context="notebook", style="white")
sb.lmplot('X1', 'X2', data=data2, fit_reg=False)
plt.show()

In [ ]:
def compute_centroids(X, idx, k):
# INPUT：数据X，聚类中心idx，簇的个数k
# OUTPUT：当前簇的聚类中心
# TODO：计算当前簇的聚类中心
    # STEP1：得到矩阵大小，初始化矩阵
    m, n = X.shape
    centroids = np.zeros((k, n))
    
    # STEP2：计算聚类中心
    # your code here  (appro ~ 2 lines)
    for i in range(k):
        indices = None
        centroids[i,:] = None
    
    return centroids

In [ ]:
compute_centroids(X, idx, 3)

如果你的答案正确，这里的输出应该是：
array([[ 2.42830111,  3.15792418],
       [ 5.81350331,  2.63365645],
       [ 7.11938687,  3.6166844 ]])
       
 

此输出也符合练习中的预期值。 
下一部分涉及实际运行该算法的一些迭代次数和可视化结果。 
这个步骤是由于并不复杂，我将从头开始构建它。 为了运行算法，我们只需要在将样本分配给最近的簇并重新计算簇的聚类中心。

In [ ]:
def run_k_means(X, initial_centroids, max_iters):
# INPUT：数据X，聚类中心idx，簇的个数k
# OUTPUT：当前簇的聚类中心
# TODO：计算当前簇的聚类中心
    # STEP1：得到矩阵大小，初始化矩阵级变量
    m, n = X.shape
    k = initial_centroids.shape[0]
    idx = np.zeros(m)
    centroids = initial_centroids
    # STEP2：实施聚类算法，调用之前的两个函数
    # your code here  (appro ~ 2 lines)    
    for i in range(max_iters):
        idx = None
        centroids = None
    
    return idx, centroids

In [ ]:
idx, centroids = run_k_means(X, initial_centroids, 10)

In [ ]:
cluster1 = X[np.where(idx == 0)[0],:]
cluster2 = X[np.where(idx == 1)[0],:]
cluster3 = X[np.where(idx == 2)[0],:]

fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(cluster1[:,0], cluster1[:,1], s=30, color='r', label='Cluster 1')
ax.scatter(cluster2[:,0], cluster2[:,1], s=30, color='g', label='Cluster 2')
ax.scatter(cluster3[:,0], cluster3[:,1], s=30, color='b', label='Cluster 3')
ax.legend()
plt.show()

我们跳过的一个步骤是初始化聚类中心的过程。 这可以影响算法的收敛。 我们的任务是创建一个选择随机样本并将其用作初始聚类中心的函数。

In [ ]:
def init_centroids(X, k):
    m, n = X.shape
    centroids = np.zeros((k, n))
    idx = np.random.randint(0, m, k)
    
    for i in range(k):
        centroids[i,:] = X[idx[i],:]
    
    return centroids

In [ ]:
init_centroids(X, 3)

我们的下一个任务是将K-means应用于图像压缩。 从下面的演示可以看到，我们可以使用聚类来找到最具代表性的少数颜色，并使用聚类分配将原始的24位颜色映射到较低维的颜色空间。 

下面是我们要压缩的图像。

In [ ]:
from IPython.display import Image
Image(filename='data/bird_small.png')

The raw pixel data has been pre-loaded for us so let's pull it in.

In [ ]:
image_data = loadmat('data/bird_small.mat')
image_data

In [ ]:
A = image_data['A']
A.shape

现在我们需要对数据应用一些预处理，并将其提供给K-means算法。

In [ ]:
# 归一化数据
A = A / 255.

# 重置矩阵大小
X = np.reshape(A, (A.shape[0] * A.shape[1], A.shape[2]))
X.shape

In [ ]:
# 随机初始化聚类中心
initial_centroids = init_centroids(X, 16)

# 运行之前写好的聚类算法
idx, centroids = run_k_means(X, initial_centroids, 10)

# 得到最后一个聚类中心
idx = find_closest_centroids(X, centroids)

# 把每一个像素值与聚类结果进行匹配
X_recovered = centroids[idx.astype(int),:]
X_recovered.shape

In [ ]:
# reshape to the original dimensions
X_recovered = np.reshape(X_recovered, (A.shape[0], A.shape[1], A.shape[2]))
X_recovered.shape

In [ ]:
plt.imshow(X_recovered)
plt.show()

您可以看到我们对图像进行了压缩，但图像的主要特征仍然存在。 这就是K-means。 下面我们来用scikit-learn来实现K-means。

In [ ]:
from skimage import io

pic = io.imread('data/bird_small.png') / 255.
io.imshow(pic)
plt.show()

In [ ]:
pic.shape

In [ ]:
# 重置图像大小
data = pic.reshape(128*128, 3)

In [ ]:
data.shape

In [ ]:
from sklearn.cluster import KMeans#导入k-means库

model = KMeans(n_clusters=16, n_init=100, n_jobs=-1)

In [ ]:
model.fit(data)

In [ ]:
centroids = model.cluster_centers_
print(centroids.shape)

C = model.predict(data)
print(C.shape)

In [ ]:
centroids[C].shape

In [ ]:
compressed_pic = centroids[C].reshape((128,128,3))

In [ ]:
fig, ax = plt.subplots(1, 2)
ax[0].imshow(pic)
ax[1].imshow(compressed_pic)
plt.show()

## Principal component analysis（主成分分析）

PCA是在数据集中找到“主成分”或最大方差方向的线性变换。 它可以用于降维。 在本练习中，我们首先负责实现PCA并将其应用于一个简单的二维数据集，以了解它是如何工作的。 我们从加载和可视化数据集开始。

In [ ]:
data = loadmat('data/ex7data1.mat')
data

In [ ]:
X = data['X']

fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(X[:, 0], X[:, 1])
plt.show()

PCA的算法相当简单。 在确保数据被归一化之后，输出仅仅是原始数据的协方差矩阵的奇异值分解。
Tip：矩阵奇异值分解可以使用np.linalg.svd(X)函数，其中X是待分解矩阵。

In [ ]:
def pca(X):
# INPUT：数据X
# OUTPUT：矩阵U，S，V
# TODO：对数据进行奇异值分解
    # STEP1：归一化数据
    # your code here  (appro ~ 1 lines)
    X = None
    
    # STEP2：计算协方差矩阵
    # your code here  (appro ~ 2 lines)  
    X = None
    cov = None
    
    # STEP3：进行奇异值分解
    # your code here  (appro ~ 1 lines)  
    U, S, V = None
    
    return U, S, V

In [ ]:
U, S, V = pca(X)
U, S, V

现在我们有主成分（矩阵U），我们可以用这些来将原始数据投影到一个较低维的空间中。 对于这个任务，我们将实现一个计算投影并且仅选择顶部K个分量的函数，有效地减少了维数。

In [ ]:
def project_data(X, U, k):
    U_reduced = U[:,:k]
    return np.dot(X, U_reduced)

In [ ]:
Z = project_data(X, U, 1)
Z

我们也可以通过反向转换步骤来恢复原始数据。

In [ ]:
def recover_data(Z, U, k):
    U_reduced = U[:,:k]
    return np.dot(Z, U_reduced.T)

In [ ]:
X_recovered = recover_data(Z, U, 1)
X_recovered

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(list(X_recovered[:, 0]), list(X_recovered[:, 1]))
plt.show()

请注意，第一主成分的投影轴基本上是数据集中的对角线。 当我们将数据减少到一个维度时，我们失去了该对角线周围的变化，所以在我们的再现中，一切都沿着该对角线。

我们在此练习中的最后一个任务是将PCA应用于脸部图像。 通过使用相同的降维技术，我们可以使用比原始图像少得多的数据来捕获图像的“本质”。

In [ ]:
faces = loadmat('data/ex7faces.mat')
X = faces['X']
X.shape

In [ ]:
def plot_n_image(X, n):
    """ plot first n images
    n has to be a square number
    """
    pic_size = int(np.sqrt(X.shape[1]))
    grid_size = int(np.sqrt(n))

    first_n_images = X[:n, :]

    fig, ax_array = plt.subplots(nrows=grid_size, ncols=grid_size,
                                    sharey=True, sharex=True, figsize=(8, 8))

    for r in range(grid_size):
        for c in range(grid_size):
            ax_array[r, c].imshow(first_n_images[grid_size * r + c].reshape((pic_size, pic_size)))
            plt.xticks(np.array([]))
            plt.yticks(np.array([]))


练习代码包括一个将渲染数据集中的前100张脸的函数。 而不是尝试在这里重新生成，您可以在练习文本中查看他们的样子。 我们至少可以很容易地渲染一个图像。

In [ ]:
face = np.reshape(X[3,:], (32, 32))

In [ ]:
plt.imshow(face)
plt.show()

看起来很糟糕。 这些只有32 x 32灰度的图像（它也是侧面渲染，但我们现在可以忽略）。 我们的下一步是在面数据集上运行PCA，并取得前100个主要特征。

In [ ]:
U, S, V = pca(X)
Z = project_data(X, U, 100)

现在我们可以尝试恢复原来的结构并再次渲染。

In [ ]:
X_recovered = recover_data(Z, U, 100)
face = np.reshape(X_recovered[3,:], (32, 32))
plt.imshow(face)
plt.show()

请注意，我们失去了一些细节，尽管没有像您预期的维度数量减少10倍。
